# Decorators and Closures

- let us mark functions
- enhances behavior of functions

We also need to understand closures before we can master decorators:

- How Python decides wheather a variable is local
- Why closures exist and how they work

A decorator is a function that takes another function as argument.

A decorator may perform some processing with the decorated Function and returns 
it or replaced it with another fnction object.

The following code:

In [1]:
def decorate(func):
    func()
    def inner():
        print('running inner()')
    return inner   # returns its inner function object

In [2]:
@decorate #target is decorated
def target():
    print('running target()')
    
target() # invoking the decorated target actually runs inner

running target()
running inner()


has the same effect as writing this:

In [3]:
def target():
    print('running target()')
    
target = decorate(target)
target()

running target()
running inner()


Strictly speaking decorators are just syntactic sugar.

- decorators are functions




Decorators usually define an inner function and return  it to replace the decorated function. 
Code that uses inner functions almost always depends on closures.

To understand closures , we need to take a step back and review how variable scopes work in Python: 

In [4]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

Since b is global and not defined a NameError was raised.


In [5]:
b = 6
f1(3)

3
6


Now let's see an example that may surprise you:


In [6]:
b = 6
def f2(a):
    print(a)
    print(b)    #this statement never runs
    b = 9
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

When Python compiles the body of the function , it decides that b is a local variable because it is assigned within the function. 

Python does not require you to declare variables, but assumes that a variable assigned in the body of a function 
is local. 

If we want the interpreter to treat b here as a global variable, we use the global declaration:

In [7]:
b = 6
def f2(a):
    global b
    print(a)
    print(b)    #this statement never runs
    b = 9
f2(3)

3
6


We have seen two  scopes in action:

*The module global scope*
    Made of names assigned to values outside of any class or function block.
    
*The f3 function local scope*
    Made of names assigned to values as parameters, or directly in the body of the function.

## Closures

A closure is a function *f* with an extended scope that includes variables referenced in the body of *f* that are not global or local variables of *f*. 

It can access nonglobal variables defined outside funtions *f* body.

Consider an *avg* function that computes the mean of an continuously growing series of values.


In [8]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

When invoked, *make_average* returns an *average* function object.

In [9]:
avg = make_averager()
avg(10)

10.0

In [10]:
avg(11)

10.5

In [11]:
avg(15)

12.0

Where does the *avg* function find the series? 

Note that **series** is a local variable of *make_averager* because the assignment **series** = [] happens in the body of that function. But when *avg(10)* is called, *make_averager* has already returned, and its local scope is long gone.

Within *averager*, **series** is a *free variable*.

![closure](free_var.png)
Figure 1. *The closure of the average extends the scope of that function to include the binding for the free variable series.*

We can the return function that we called *avg* by looking at the __ code__ attribute. __ code__ represents the compiled body of the function.

In [12]:
avg.__code__.co_varnames

('new_value', 'total')

In [13]:
avg.__code__.co_freevars

('series',)

In [14]:
avg.__closure__

(<cell at 0x7f161dd309a0: list object at 0x7f161d505c40>,)

In [15]:
avg.__closure__[0].cell_contents

[10, 11, 15]

The body of a function is evaluated in the environment where the function is defined, not the environment
where the function is called. [lexical scope](https://courses.cs.washington.edu/courses/cse341/17au/unit3notes.pdf)
The scope where a object was defined we also call *lexical* scope.

A closure is a function that retains/keeps the bindings of the free variables that exists when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.
    

Let's try to make *make_averager* more efficient:

In [16]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

*count* and *total* are immutable and all you can do is read, never update. If you try to rebind them, as in *count = count + 1*, then you are implicitly creating a local variable *count*. Therefore, it's not longer a free variable, but a local variable and will be not saved in the closure.

To declare variabl as free variable us the **nonlocal** keyword:

In [17]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

10.0

# Exercises 1

- look at the following functions:

In [183]:
def triple():
    a = 3
    def multiply(b):
        return a * b
    return multiply
triple()(6)



18

- what is the free variable?
- what is the closure?
- use __ code__.co_varnames, __ code__.co_freevars, and __ closure__[0].cell_contents

In [184]:
curry = triple()


In [186]:
curry.__code__.co_varnames, curry.__code__.co_freevars, curry.__closure__[0].cell_contents

(('b',), ('a',), 3)

# Exercise 2

- create a function double analogous to the triple triple
- what is the free variable?
- what is the closure?
- use __ code__.co_varnames, __ code__.co_freevars, and __ closure__[0].cell_contents

In [187]:
def double():
    a = 2
    def multiply(b):
        return a * b
    return multiply

double()(6) ## 12

12

# Exercise 3

- look at the following function:

In [191]:
def deco(func):
    def inner(a, b):
        return 3 * func(a,b)
    return inner

def multiply(a,b):
    return a * b

deco(multiply)(1,3)



9

- what is the free variable?
- what is the closure?
- use __ code__.co_varnames, __ code__.co_freevars, and __ closure__[0].cell_contents

In [192]:
def multiply(a, b):
    return a*b


6

## Implementing a Simple Decorator



In [148]:
import time

def clock(fun):
    def clocked(n):  #### inner function
        t0 = time.time()
        result = fun(n)  ### fun is the free variable of the closure
        elapsed = time.time() - t0
        #print(f"{elapsed:0.8f}s func({n}) --> {result}")
        print("{elapsed: 0.8f}s func({n}) --> {result}".format(elapsed=elapsed, n=n, result=result))
        return result
    return clocked ## return the inner function to replace the decorated function

In [149]:

def snooze(s):
    time.sleep(s)
    


In [150]:
snooze = clock(snooze)


In [151]:
snooze(.5)

 0.50059819s func(0.5) --> None


This is same as writing:

In [33]:
@clock
def snooze(s):
    time.sleep(s)

In [34]:
snooze(.5)

0.50062108s func(0.5) --> None


In [28]:

def fac(n):
    result = 1
    while n >= 1:
        result = result * n
        n -= 1
    return result

In [29]:
fac = clock(fac)


In [32]:
fac(6)

0.00000648s func(6) --> 720


720

This is same as writing:

In [35]:
@clock 
def fac(n):
    result = 1
    while n >= 1:
        result = result * n
        n -= 1
    return result

In [36]:
fac(6)

0.00000215s func(6) --> 720


720

- the decorator replaces the decorated function with a new function (inner)
- it accepts the same arguments and 
- (usually) returns whatever the decorated function was suppose to return
- does extra processing
- attaches additional responsibilties dynamically

## Stacked Decorators

To make sense of stacked decorators, recall that @ is syntax sugar for applying the decorator function to the function below it. If there's more than one decorator, they behave like nested function calls. This:

In [ ]:
@alpha
@beta
def my_fn():

is the same as this:

```my_fn = alpha(beta(my_fn))```

In other words, the *beta* decorator is applied first, and the function it returns is then passed to *alpha*.

In [6]:
def alpha(func):
    def inner():
        return 'alpha ' + func()
    return inner

def beta(func):
    def inner():
        return 'beta ' + func()
    return inner

In [7]:
@alpha
@beta
def test():
    return 'Hello World'

In [8]:
test()

'alpha beta Hello World'

## Unpacking and decorators

To make your decorator function more general, use unpacking:

In [14]:
def decorator(func):
    def inner(*args, **kwargs):
        return 'deco ' + func(*args, **kwargs)
    return inner

In [17]:
@decorator
def test(w1, w2='world'):
    return f'{w1} {w2}'

In [18]:
test('Hello')

'deco Hello world'

## Parameterized Decorator

To pass other parameters than the function to the decorator we need to make a **decorator factory** that takes those arguments and returns a decorator

In [152]:
def wrap_with(tag='***'):
    """Wrap the text in an HTML tag."""
    def decorator(func):
        def inner(first, last):
            return f"{tag} {func(first, last)} {tag}"
        return inner
    return decorator

In [153]:
@wrap_with('###')
def get_full_name(first, last):
    """Return the full name of a person."""
    return f"{first} {last}"

In [154]:
get_full_name('piet', 'pro')

'### piet pro ###'

Same as:

In [155]:
def get_full_name(first, last):
    """Return the full name of a person."""
    return f"{first} {last}"

get_full_name = wrap_with('###')(get_full_name)

In [156]:
get_full_name('piet', 'pro')

'### piet pro ###'

In [161]:
@wrap_with()
def get_full_name(first, last):
    """Return the full name of a person."""
    return f"{first} {last}"

In [162]:
get_full_name('piet', 'pro')

'*** piet pro ***'

Notice that you always need to call the decorator factory. That factory returns the decorator.
Same as:

In [163]:
decorater = wrap_with()
@decorater
def get_full_name(first, last):
    """Return the full name of a person."""
    return f"{first} {last}"

In [164]:
get_full_name('piet', 'pro')

'*** piet pro ***'

You also can run multiple decorators:

In [158]:
@wrap_with('###')
@wrap_with('---')
def get_full_name(first, last):
    """Return the full name of a person."""
    return f"{first} {last}"

In [159]:
get_full_name('piet', 'teacher')

'### --- piet teacher --- ###'

Same as:

In [169]:
def get_full_name(first, last):
    """Return the full name of a person."""
    return f"{first} {last}"

In [170]:
get_full_name = wrap_with('###')(wrap_with('---')(get_full_name))

In [171]:
get_full_name('piet', 'teacher')

'### --- piet teacher --- ###'

Let's return to our clock decorator:

In [173]:
DEFAULT_FMT = "{elapsed: 0.8f}s func(n) --> {result}"

def clock(fmt=DEFAULT_FMT):
    def decorator(fun):
        def clocked(n):  #### inner function
            t0 = time.time()
            result = fun(n)  ### fun is the free variable of the closure
            elapsed = time.time() - t0
            print("{elapsed: 0.8f}s func({n}) --> {result}".format(elapsed=elapsed, n=n, result=result))
            print(fmt.format(elapsed=elapsed, n=n, result=result))
            #print(fmt.format(**locals()))
            #print({**locals()})
            return result
        return clocked
    return decorator ## return the inner function to replace the decorated function

In [174]:
x = 1
y = 2

"{x} and {y}".format(x=x, y=y)

'1 and 2'

In [175]:
@clock()
def hello(n):
    return 'hello world'

In [176]:
hello(1)


 0.00000238s func(1) --> hello world
 0.00000238s func(n) --> hello world
 0.00000238s func(n) --> hello world


'hello world'

In [177]:
@clock(fmt="{elapsed}s : {result}")
def hello(n):
    return 'hello world'

In [147]:
hello(1)

 0.00000334s func(1) --> hello world
3.337860107421875e-06s : hello world
3.337860107421875e-06s : hello world


'hello world'